In [1]:
drive_path = '/Volumes/Brain2016 1'

%gui qt

import os
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import pandas as pd
import scipy.ndimage as ndi
import numpy as np
import time
import get_connectivity as gc

import pyqtgraph as pg
import pyqtgraph.metaarray as metaarray
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph.opengl as pgl
pg.mkQApp()


In [2]:
# When downloading 3D connectivity data volumes, what resolution do you want (in microns)?  
# Options are: 10, 25, 50, 100
resolution_um=25

# The manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives.
# If you supply a relative path, it is assumed to be relative to your
# current working directory.
manifest_file = os.path.join(drive_path, "MouseConnectivity","manifest.json")

mcc = MouseConnectivityCache(manifest_file=manifest_file, resolution=resolution_um)

template, template_info = mcc.get_template_volume()

ontology = mcc.get_ontology()

# Define the visualization function

In [64]:
# def vis3D(brain_array,inj_array,pad = 30,ds_factor=6):
    
#     # set up time variables
#     now = time.time()
#     now_start = now
    
#     # initialize the window
#     view = pgl.GLViewWidget()
#     #view.setBackgroundColor('w') # maybe?     
    
#     # downsample the brain image using the ds_factor
#     img = brain_array[::ds_factor,::ds_factor,::ds_factor]
    
#     # do padding of the brain to avoid holes during rendering
#     pad_img = np.zeros((img.shape[0]+pad, img.shape[1]+pad, img.shape[2]+pad), dtype=img.dtype)
#     pad_img[pad/2:pad/2+img.shape[0], pad/2:pad/2+img.shape[1], pad/2:pad/2+img.shape[2]] = img
#     print "padding %0.2f" % (time.time() - now); now = time.time() 
    
#     # build the brain isosurface
#     verts, faces = pg.isosurface(ndi.gaussian_filter(pad_img.astype('float32'), (2, 2, 2)), 5.0)
#     md = pgl.MeshData(vertexes=verts, faces=faces)
#     mesh = pgl.GLMeshItem(meshdata=md, smooth=True, color=[0.5, 0.5, 0.5, 0.1], shader='balloon')
#     mesh.setGLOptions('additive')
#     mesh.translate(-pad_img.shape[0]/2., -pad_img.shape[1]/2., -pad_img.shape[2]/2.)
#     mesh.rotate(-90, 1, 0, 0)
#     view.addItem(mesh)
#     print "build brain isosurface %0.2f" % (time.time() - now); now = time.time() 
    
#     # render the injection(s) as a volume
#     # inj_array should be a list of tuples, with the first element in the tuple
#     # being the plotting color (a RGB value), and the second element being the 
#     # ND-array of the volumetric data for a given injection
#     vols = np.zeros(inj_array[0][1].shape + (4,), dtype='float32')
#     for inj in range(len(inj_array)):
#         col = inj_array[inj][0]
#         vols[...,0] += col[0] * inj_array[inj][1] # red channel
#         vols[...,1] += col[1] * inj_array[inj][1] # green channel
#         vols[...,2] += col[2] * inj_array[inj][1] # blue channel
#         vols[...,3] += inj_array[inj][1] * 255 # alpha channel
#     print "sum the projections %0.2f" % (time.time() - now); now = time.time()
    
    
#     # Now divide all color channels by a matrix that contains the max 
#     # color value across the channels at each voxel.
#     # Then, multiply it all by 255. 
#     # this will make sure that each voxel, if it has any content, will
#     # have at least one color channel that is at full value (255)
#     # we do this to avoid having dark colors displayed over the top of bright colors
#     #vols[...,0:3] = (vols[...,0:3]/np.rollaxis(np.tile(vols[...,0:3].max(axis=3),(3,1,1,1)),0,4)) * 10
#     # make sure the maximum alpha is 255
#     vols[...,3] *= 5
#     vols[...,3] = np.clip(vols[...,3],0,255)
#     print "clean up the projections %0.2f" % (time.time() - now); now = time.time() 
    
#     # now add the volume to the view window
#     vi = pgl.GLVolumeItem(vols)
#     vi.translate(-vols.shape[0]/(2.*ds_factor), -vols.shape[1]/(2.*ds_factor), -vols.shape[2]/(2.*ds_factor))
#     vi.scale(1./ds_factor,1./ds_factor,1./ds_factor)
#     vi.setGLOptions('additive')
#     vi.rotate(-90, 1, 0, 0)
#     view.addItem(vi)
#     print "add volume to window %0.2f" % (time.time() - now); now = time.time() 
#     view.setCameraPosition(distance=200, elevation=20, azimuth=90)
#     view.show()
    
#     print "show rendering %0.2f" % (time.time() - now); now = time.time() 
#     print "total run time: %0.2f" % (time.time() - now_start)
#     return view




In [87]:
def vis3D(brain_array,inj_array,pad = 30,ds_factor=6):
    
    # set up time variables
    now = time.time()
    now_start = now
    
    view = vis3D_glassBrain(template,pad,ds_factor)
    print "build brain isosurface %0.2f" % (time.time() - now); now = time.time() 
    
    view = vis3D_projections(view,inj_array,ds_factor)
    print "build injection volume %0.2f" % (time.time() - now); now = time.time() 
    
    view.show()
    
    print "rendering %0.2f" % (time.time() - now); now = time.time() 
    print "total run time: %0.2f" % (time.time() - now_start)
    
    return view

In [88]:
def vis3D_glassBrain(brain_array,pad,ds_factor):
    
    # initialize the window
    view = pgl.GLViewWidget()   
    
    # downsample the brain image using the ds_factor
    img = brain_array[::ds_factor,::ds_factor,::ds_factor]
    
    # do padding of the brain to avoid holes during rendering
    pad_img = np.zeros((img.shape[0]+pad, img.shape[1]+pad, img.shape[2]+pad), dtype=img.dtype)
    pad_img[pad/2:pad/2+img.shape[0], pad/2:pad/2+img.shape[1], pad/2:pad/2+img.shape[2]] = img
    
    # build the brain isosurface
    verts, faces = pg.isosurface(ndi.gaussian_filter(pad_img.astype('float32'), (2, 2, 2)), 5.0)
    md = pgl.MeshData(vertexes=verts, faces=faces)
    mesh = pgl.GLMeshItem(meshdata=md, smooth=True, color=[0.5, 0.5, 0.5, 0.1], shader='balloon')
    mesh.setGLOptions('additive')
    mesh.translate(-pad_img.shape[0]/2., -pad_img.shape[1]/2., -pad_img.shape[2]/2.)
    mesh.rotate(-90, 1, 0, 0)
    view.addItem(mesh)
    view.setCameraPosition(distance=200, elevation=20, azimuth=90)
    view.show()

    return view

In [89]:
def vis3D_projections(view,inj_array,ds_factor):
    
    # render the injection(s) as a volume
    # inj_array should be a list of tuples, with the first element in the tuple
    # being the plotting color (a RGB value), and the second element being the 
    # ND-array of the volumetric data for a given injection
    vols = np.zeros(inj_array[0][1].shape + (4,), dtype='float32')
    for inj in range(len(inj_array)):
        col = inj_array[inj][0]
        vols[...,0] += col[0] * inj_array[inj][1] # red channel
        vols[...,1] += col[1] * inj_array[inj][1] # green channel
        vols[...,2] += col[2] * inj_array[inj][1] # blue channel
        vols[...,3] += inj_array[inj][1] * 255    # alpha channel

    # Set alpha and make sure the maximum alpha is 255
    vols[...,3] *= 5
    vols[...,3] = np.clip(vols[...,3],0,255)

    # now add the volume to the view window
    vi = pgl.GLVolumeItem(vols)
    vi.translate(-vols.shape[0]/(2.*ds_factor), -vols.shape[1]/(2.*ds_factor), -vols.shape[2]/(2.*ds_factor))
    vi.scale(1./ds_factor,1./ds_factor,1./ds_factor)
    vi.setGLOptions('additive')
    vi.rotate(-90, 1, 0, 0)
    view.setCameraPosition(distance=200, elevation=20, azimuth=90)
    view.addItem(vi)
    
    return view

In [4]:
fid_PRNc_III, fpd_PRNc_III=gc.get_connectivity('PRNc','III',ontology,mcc)
piIDs = list(fpd_PRNc_III.experiment_id)

In [5]:
fid_SCm_PRNc, fpd_SCm_PRNc=gc.get_connectivity('SCm','PRNc',ontology,mcc)
spIDs = list(fpd_SCm_PRNc.experiment_id)

In [68]:
# GET PROJECTION DENSITIES
pdens1,_ = mcc.get_projection_density(158838128)
pdens2,_= mcc.get_projection_density(175158132)

In [81]:
# DEFINE THE INJECTION INPUT(S)
# lt = [([255,15,0],pdens1),([0,15,255],pdens2)]
lt = [([0,15,240],pdens2)]


# Note that the dominant RGB color should be 255, and the added color you want at the peak intensity
# (i.e., where the injection occurred) should be set to about 10 or 15.

In [90]:
# RUN THE ENTIRE VISUALIZATION
view = vis3D(template,lt)

build brain isosurface 1.70
build injection volume 13.77
rendering 0.00
total run time: 15.46


In [ ]:
# JUST PLOT THE GLASS BRAIN
view = vis3D_glassBrain(template,pad=30,ds_factor=6)

In [91]:
# REMOVE ALL ITEMS FROM VIEW
for i in range(len(view.items)):
    view.removeItem(view.items[0])  
# view.items

In [79]:
# ROTATE THE IMAGE 
for i in range(360): # determine the number of steps
    view.orbit(-2,0)  # determine the step size in horizontal and vertical dimensions
    time.sleep(0.1) # determine the time between steps
    pg.QtGui.QApplication.processEvents()

In [ ]:
# import sys
# print sys.path
# sys.path.append('somepath')



# list of tuples
# lt = [('r',np.ones((5,5))),('g',np.ones((8,2)))]
# print(lt[0][1])



# render volume
#vol = np.empty(img.shape + (4,), dtype='ubyte')
#vol[:] = img[..., None]
#vol = np.ascontiguousarray(vol.transpose(1, 2, 0, 3))
#vi = pgl.GLVolumeItem(vol)
#self.glView.addItem(vi)
#vi.translate(-vol.shape[0]/2., -vol.shape[1]/2., -vol.shape[2]/2.)




# remove item
# view.removeItem(mesh)


# g = pgl.GLGridItem()
# g.scale(10, 10, 1)
# view.addItem(g)


# # render volume
# vol = np.zeros(pdensity.shape + (4,), dtype='ubyte')
# vol[...,3] = pdensity*255
# vol[...,1] = 255
# #vol = np.ascontiguousarray(vol.transpose(1, 2, 0, 3))
# vi = pgl.GLVolumeItem(vol)
# vi.translate(-vol.shape[0]/8., -vol.shape[1]/8., -vol.shape[2]/8.)
# vi.scale(1./4,1./4,1./4)



# verts_pd, faces_pd = pg.isosurface(ndi.gaussian_filter(pd.astype('float32'), (2, 2, 2)), .05)
# md_d = pgl.MeshData(vertexes=verts_pd, faces=faces_pd)
# mesh_d = pgl.GLMeshItem(meshdata=md_d, smooth=True, color=[1, 0.5, 1, 0.9], shader='balloon')
# mesh_d.setGLOptions('additive')
# mesh_d.translate(-pd.shape[0]/2., -pd.shape[1]/2., -pd.shape[2]/2.)
# view.addItem(mesh_d)